<a href="https://colab.research.google.com/github/retroam/harmonizer/blob/develop/ocean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install pycountry -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 49.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import pandas as pd
import os

In [11]:
fl = 'OpenClimate/harmonize/data/raw/BP_review/bp-stats-review-2022-all-data.xlsx'
fl = os.path.abspath(fl)

In [16]:
def df_wide_to_long(df=None,
                    value_name=None,
                    var_name=None):

    # set default values (new column names)
    # new column name with {value_vars}
    var_name = "year" if var_name is None else var_name
    # new column name with values
    value_name = "values" if value_name is None else value_name

    # ensure correct type
    assert isinstance(df, pd.core.frame.DataFrame), f"df must be a DataFrame"
    assert isinstance(var_name, str), f"var_name must a be string"
    assert isinstance(value_name, str), f"value_name must be a string"

    # ensure column names are strings
    df.columns = df.columns.astype(str)

    # columns to use as identifiers (columns that are not number)
    id_vars = [val for val in list(df.columns) if not val.isdigit()]

    # columns to unpivot (columns that are numbers)
    value_vars = [val for val in list(df.columns) if val.isdigit()]

    # Unpivot (melt) a DataFrame from wide to long format
    df_long = df.melt(id_vars=id_vars,
                      value_vars=value_vars,
                      var_name=var_name,
                      value_name=value_name)

    # convert var_name column to int
    df_long[var_name] = df_long[var_name].astype(int)

    return df_long

In [17]:
raw = pd.ExcelFile(fl)

In [21]:
import pycountry
from typing import List
from typing import Dict


def country_lookup(name):
    try:
        return pycountry.countries.lookup(name).alpha_2
    except LookupError:
        return float('NaN')

In [22]:
xl = pd.ExcelFile(fl)
sheets = xl.sheet_names

replace_dict = {
    'Trinidad & Tobago': 'Trinidad and Tobago',
    'China Hong Kong SAR': 'China',
    'Iran': 'Iran, Islamic Republic of'
}

drop_columns = ['2021.1', '2011-21', '2021.2']
rename_columns = {'Million tonnes of carbon dioxide equivalent': 'country'}
country_groups = ['Central America', 'Eastern Africa', 'Middle Africa', 'Western Africa']
not_countries = ["Source:", "Notes:", "Growth", "Data ", "European Union", "OECD", "0.05%", "Other "]

df = (
    pd.read_excel(fl, sheet_name='CO2e Emissions', header=2)
    .drop(columns = drop_columns)
    .rename(columns = rename_columns)
    .loc[lambda x: x['country'].notnull()]
    .loc[lambda x: ~x['country'].str.contains('total', case=False)]
    .loc[lambda x: ~x['country'].isin(country_groups)]
    .loc[lambda x:  ~(x['country'].str.contains('|'.join(not_countries)))]
    .assign(country = lambda x: x.country.replace(replace_dict))
)

# reshape dataframe
df = df_wide_to_long(df, value_name='emissions', var_name='year')

# convert from million metric tones to metric tonnes
df['total_emissions'] = df['emissions'].apply(lambda x: x * 10**6)

# get actor_id from country name
df_iso = pd.DataFrame(
    data=[(name, country_lookup(name)) for name in set(list(df.country))],
    columns=['country', 'actor_id']
)

if not_found:=list(df_iso.loc[df_iso['actor_id'].isnull(), 'country']):
    print(f"Countries not found: {not_found}")

In [23]:
df

,country,year,emissions,total_emissions
0,Canada,1990,480.394916,4.803949e+08
1,Mexico,1990,315.852733,3.158527e+08
2,US,1990,5275.397531,5.275398e+09
3,Argentina,1990,123.904223,1.239042e+08
4,Brazil,1990,217.067921,2.170679e+08
...,...,...,...,...
2235,Philippines,2021,149.473427,1.494734e+08
2236,South Korea,2021,628.697163,6.286972e+08
2237,Taiwan,2021,285.258943,2.852589e+08
2238,Thailand,2021,301.774696,3.017747e+08
